# Bitcoin Sentiment Analysis

This notebook demonstrates the BTC sentiment analysis pipeline.

In [1]:
import sys
sys.path.append('../src')

from btc_sentiment.pipelines.ingest_pipeline import run_ingest_pipeline
from btc_sentiment.utils.io import load_daily_sentiment
from btc_sentiment.utils.viz import plot_daily_sentiment

## Run Pipeline

In [2]:
%pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

daily_sentiment = run_ingest_pipeline(
    days_back=7,
    tweets_per_day=0,
    messages_per_channel=100
)

print(f"Processed {len(daily_sentiment)} daily records")

Note: you may need to restart the kernel to use updated packages.
Error fetching from Market Mercenaries: Cannot find any entity corresponding to "Market Mercenaries"


KeyError: 'date'

## Visualize Results

In [5]:
data = load_daily_sentiment()

if data:
    plot_daily_sentiment(data)
else:
    print("No data found. Run the pipeline first.")

No data found. Run the pipeline first.


## Enhanced Analysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

data = load_daily_sentiment()
if data:
    df = pd.DataFrame([r.dict() for r in data])
    df['date'] = pd.to_datetime(df['date'])
    print(f"Loaded {len(df)} daily sentiment records")
    display(df)

In [ ]:
if not df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Sentiment score distribution
    for source in df['source'].unique():
        source_data = df[df['source'] == source]
        axes[0,0].hist(source_data['avg_score'], alpha=0.7, label=source, bins=20)
    axes[0,0].set_title('Sentiment Score Distribution')
    axes[0,0].legend()
    
    # Message count by source
    source_counts = df.groupby('source')['count'].sum()
    axes[0,1].pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%')
    axes[0,1].set_title('Message Volume by Source')
    
    # Sentiment labels
    label_counts = df['label'].value_counts()
    colors = {'positive': 'green', 'neutral': 'gray', 'negative': 'red'}
    bar_colors = [colors.get(label, 'blue') for label in label_counts.index]
    axes[1,0].bar(label_counts.index, label_counts.values, color=bar_colors)
    axes[1,0].set_title('Sentiment Label Distribution')
    
    # Daily trend
    for source in df['source'].unique():
        source_data = df[df['source'] == source].sort_values('date')
        axes[1,1].plot(source_data['date'], source_data['avg_score'], marker='o', label=source)
    axes[1,1].set_title('Daily Sentiment Trend')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()